# Does Enhancing Images also Enhance the Outcome of Image Recognition of Neural Networks

# General Layout for this program

1. Imports
2. Load the training set
3. Load in the test set
4. Load in the test set labels
5. Create a copy of the image set and apply CLAHE to it
6. Load in 2 pretrained resnets
7. Train resnet1 on OG images, train resnet2 on CLAHE images
8. Compare performance of both

# Imports

In [43]:
import torch
from torchvision import *
from torch import optim
import torch.nn as nn
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
import os
from PIL import Image
from timeit import default_timer as timer
import cv2
from torch.utils.data import Dataset, DataLoader


# Import the model

In [44]:
model = models.resnet18(pretrained=True)   #load resnet18 model
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, 2) #(num_of_class == 2)

/Users/ian/miniforge3/envs/ml/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/ian/miniforge3/envs/ml/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Organize ImageNet so there are 50 images from each catagory in a test folder

-> All under imageNet/ILSVRC/Data/CLS-LOC/test

In [58]:
from asyncio import constants
import shutil

headDirectory = './imageNet/ILSVRC/Data/CLS-LOC/'

if not os.path.isdir('./imageNet/ILSVRC/Data/CLS-LOC/' + 'test'):
    os.mkdir(headDirectory + 'test')
else:
    shutil.rmtree(headDirectory + 'test')
    os.mkdir(headDirectory + 'test')

for folder in os.listdir(headDirectory + 'train'):

    if (folder == 'sals' or folder == '.DS_Store'):
        continue
    
    f = os.path.join(headDirectory + 'train/', folder)
    os.mkdir(headDirectory + 'test/' + folder)
    counter = 0

    for file in sorted( os.listdir(f)):
        if counter == 50:
            break
        else:
            shutil.copy(f + '/' + file, headDirectory + 'test/' + folder)
            counter += 1

# Open the ImageNet Spotted Salamanders
Loop through the imagenet training directory pulling in all 1200 Spotted Salamader images. Then create a copy of the images and label it CLAHE

In [46]:
smallestHeight = 1000
smallestWidth = 1000
largestHeight = 0
largestWidth = 0
numImages = 0

directory = './imageNet/ILSVRC/Data/CLS-LOC/train/n01632458'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        numImages += 1
        img = cv2.imread(f)

        if img.shape[0] < smallestHeight:
            smallestHeight = img.shape[0]
        if img.shape[1] < smallestWidth:
            smallestWidth = img.shape[1]
        if img.shape[0] > largestHeight:
            largestHeight = img.shape[0]
        if img.shape[1] > largestWidth:
            largestWidth = img.shape[1]

print("Number of Images: " + str(numImages))
# print('Smallest Height = ' + str(smallestHeight))
# print('Smallest Width = ' + str(smallestWidth))
# print('Largest Height = ' + str(largestHeight))
# print('Largest Width = ' + str(largestWidth))

Number of Images: 1150


In [47]:
import torchvision
import numpy
from torchvision import transforms
transforms = transforms.Compose(
[
   transforms.ToTensor(),
   transforms.Resize((smallestHeight, smallestWidth)),
   transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
])

directory = 'imageNet/ILSVRC/Data/CLS-LOC/train/sals'
train_dataset = torchvision.datasets.ImageFolder(directory, transforms)
train_dataset = torch.utils.data.Subset(train_dataset, range(50, len(train_dataset)))


print(len(train_dataset))

print(type(train_dataset))
print(type([train_dataset[0]]))

1250
<class 'torch.utils.data.dataset.Subset'>
<class 'list'>


In [48]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F

def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [49]:
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [50]:
net = models.resnet18(pretrained=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 128)
device = 'cpu'


In [51]:
n_epochs = 5
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    batch_idx = -1
    for data_, target_ in train_loader:
        batch_idx += 1

        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        
        outputs = net(data_)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        net.eval()
        for data_t, target_t in (train_loader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(train_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(net.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    net.train()

Epoch 1

Epoch [1/5], Step [0/13], Loss: 4.5263

train-loss: 4.5532, train-acc: 0.8800
validation loss: 4.6035, validation acc: 0.4800

Improvement-Detected, save-model
Epoch 2

Epoch [2/5], Step [0/13], Loss: 4.5412

train-loss: 4.5537, train-acc: 1.3600
validation loss: 4.5872, validation acc: 0.8800

Improvement-Detected, save-model
Epoch 3

Epoch [3/5], Step [0/13], Loss: 4.5567

train-loss: 4.5523, train-acc: 0.7200
validation loss: 4.5869, validation acc: 0.5600

Epoch 4

Epoch [4/5], Step [0/13], Loss: 4.5303

train-loss: 4.5495, train-acc: 0.8000
validation loss: 4.5900, validation acc: 0.5600

Epoch 5

Epoch [5/5], Step [0/13], Loss: 4.5411

train-loss: 4.5453, train-acc: 1.2800
validation loss: 4.5853, validation acc: 1.2800

Improvement-Detected, save-model


## Save model

In [52]:
save_path = 'SalamanderResnet.pth'
torch.save(model.state_dict(), save_path)

# Switch into eval mode and test it on some images

In [59]:
from cgi import test


model.eval()

testSet = []
directory = './imageNet/ILSVRC/Data/CLS-LOC/test'


#Load in all test salamader images
testData = torchvision.datasets.ImageFolder(directory, transforms)


print(type(testData))
print(len(testData))

<class 'torchvision.datasets.folder.ImageFolder'>
49999


In [61]:
#Switch Model into eval mode
model.eval()
test_loader = DataLoader(testData, batch_size=50, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    for i, (inputs, labels) in enumerate(test_loader):
        print("label = ", labels)
        print("inputs = ", inputs)
        print("loop " + str(i))
        if (i == 50):
            break
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        if i == 0:
            print('======>RESULTS<======')
            #images = torchvision.utils.make_grid(inputs[:4])
    epoch_loss = running_loss / len(testData)
    epoch_acc = running_corrects / len(testData) * 100.
    print('[Test #{}] Loss: {:.4f} Acc: {:.4f}%'.
          format(epoch, epoch_loss, epoch_acc))

#print("Rought Answer: {:.4f}%".format(roughtAnswer))

label =  tensor([763, 534, 356,  65,  82, 829, 569, 417, 285, 195,  40, 701, 800, 553,
        771, 908, 697, 951, 563, 332, 546, 351, 541, 740, 291, 274, 230, 737,
        875, 331, 982, 619, 439, 495, 149, 428, 931, 785, 700, 668, 790, 980,
        602, 813, 408, 798, 449, 529, 667, 835])
inputs =  tensor([[[[0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
          [0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
          [0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
          ...,
          [0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
          [0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
          [0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026]],

         [[0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
          [0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
          [0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
          ...,
          [0.0026, 0.0026, 0.0026,  ..., 0.0026, 0.0026, 0.0026],
      

IndexError: Target 763 is out of bounds.